<a href="https://colab.research.google.com/github/nyquist/aoe2de/blob/main/AOE2DE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Environment Setup

In [18]:
! [[ -d "aoe2de" ]] && (cd aoe2de && git pull) || git clone https://github.com/nyquist/aoe2de.git
! ls -la ./aoe2de/utils

Cloning into 'aoe2de'...
remote: Enumerating objects: 13, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 13 (delta 1), reused 2 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (13/13), 717.16 KiB | 3.90 MiB/s, done.
Resolving deltas: 100% (1/1), done.
total 12
drwxr-xr-x 2 root root 4096 Dec 14 14:36 .
drwxr-xr-x 4 root root 4096 Dec 14 14:36 ..
-rw-r--r-- 1 root root   22 Dec 14 14:36 test.py


In [23]:
import os

def list_files_with_dynamic_sizes(path="."):
    """Lists all files and their sizes with dynamic units (B, kB, MB, etc.) in the given directory.

    Args:
        path (str, optional): The directory path to list files from. Defaults to "." (current directory).
    """
    for entry in os.scandir(path):
        if entry.is_file():
            file_size = os.path.getsize(entry.path)
            unit_index = 0
            units = ["B", "kB", "MB", "GB", "TB"]
            while file_size >= 1024 and unit_index < len(units) - 1:
                file_size /= 1024
                unit_index += 1
            print(f"{entry.name}: {file_size:.2f} {units[unit_index]}")

# Call the function to list files and sizes with dynamic units in the current directory


# Download record

In [24]:
import requests

replay_url = "https://aoe.ms/replay/?gameId=357732557&profileId=18446743"  # Replace with the actual URL you copied

response = requests.get(replay_url)
response.raise_for_status()  # Raise an exception for bad responses
print (response.status_code)
content_disposition = response.headers['Content-Disposition']
import re
match = re.search(r'filename="?([^";]+)"?', content_disposition)
if match:
    file_name = match.group(1)
    print(f"Extracted filename: {file_name}")
    with open(file_name, "wb") as f:
      f.write(response.content)
else:
    print("Filename not found in Content-Disposition header.")


print(f"Replay file downloaded as {file_name}")
list_files_with_dynamic_sizes()





200
Extracted filename: AgeIIDE_Replay_357732557.zip
Replay file downloaded as AgeIIDE_Replay_357732557.zip
AgeIIDE_Replay_357732557.zip: 1.37 MB


### Unzip record file

In [25]:
zip_file_name = file_name

import zipfile

try:
    with zipfile.ZipFile(zip_file_name, 'r') as zip_ref:
        zip_ref.extractall(".")  # Extract all files to the current directory
    print(f"Successfully extracted files from {zip_file_name}")

    # Find the extracted replay file (assuming it's an .mgz file)
    extracted_file_name = None
    for filename in os.listdir("."):
        print (filename)
        if filename.endswith(".aoe2record"):
            extracted_file_name = filename
            break

    if extracted_file_name:
        # Process the extracted replay file
        print ("Extracted file:", extracted_file_name)
    else:
        print("Error: Could not find extracted replay file (.mgz).")

except zipfile.BadZipFile:
    print(f"Error: {zip_file_name} is not a valid zip file.")
except FileNotFoundError:
    print(f"Error: {zip_file_name} not found.")
except RuntimeError as e:
    if "could not parse" in str(e):
        print(f"Error parsing the replay file: {e}")
        print("The replay file might be corrupted or in an unsupported format.")
        print("Please try re-downloading the replay file or using a different one.")
    else:
        raise  # Re-raise the exception if it's not related to parsing

Successfully extracted files from AgeIIDE_Replay_357732557.zip
.config
AgeIIDE_Replay_357732557.zip
AgeIIDE_Replay_357732557.aoe2record
Extracted file: AgeIIDE_Replay_357732557.aoe2record


## Convert file to json

In [26]:
import json
! pip install mgz
from mgz.model import parse_match, serialize
f = extracted_file_name
outFile=f"{extracted_file_name}.json"
print (f)
with open(f, 'rb') as h:
    match = parse_match(h)
    # print(json.dumps(serialize(match), indent=2))
    json_object=json.dumps(serialize(match), indent=2)
    with open(outFile, "w") as o:
        o.write(json_object)

list_files_with_dynamic_sizes()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.4/80.4 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for mgz: filename=mgz-1.8.28-py3-none-any.whl size=72422 sha256=c298b7ba19617a3bb8fc6f8b848bf9b687dde8eaffc5ce8834bf91c09cd00ea5
  Stored in directory: /root/.cache/pip/wheels/71/f0/68/23ed8196623ac82c61348a1fdee66446ac501bf9a1bf635720
  Created wheel for construct: filename=construct-2.8.16-py2.py3-none-any.whl size=70258 sha256=e91a54a92c4ffccd48d3f4fbd0382726c6c61832e11df9c4408da70ee4f93e07
  Stored in directory: /root/.cache/pip/wheels/78/37/f7/3a9d487ec99b26560f68e030c537774c3ed4e4e39128c40bf7
  Created wheel for aocref: filename=aocref-2.0.20-py3-none-any.whl size=94222 sha256=ce1ff35f36dffc620fa3b09d838d999

Parse JSON

In [54]:
with open(outFile, 'r') as json_file:
    json_data = json.load(json_file)

print(json.dumps(serialize(json_data['players']), indent=2))

[
  {
    "number": 1,
    "name": "risk",
    "color": "Purple",
    "color_id": 5,
    "civilization": "Khmer",
    "civilization_id": 28,
    "position": {
      "x": 34.0,
      "y": 62.0
    },
    "objects": [
      {
        "name": "Town Center",
        "class_id": 80,
        "object_id": 109,
        "instance_id": 22046,
        "index": 0,
        "position": {
          "x": 34.0,
          "y": 62.0
        }
      },
      {
        "name": "Town Center",
        "class_id": 80,
        "object_id": 618,
        "instance_id": 22047,
        "index": 0,
        "position": {
          "x": 35.0,
          "y": 61.0
        }
      },
      {
        "name": "Town Center",
        "class_id": 80,
        "object_id": 619,
        "instance_id": 22048,
        "index": 0,
        "position": {
          "x": 33.5,
          "y": 62.5
        }
      },
      {
        "class_id": 80,
        "object_id": 1649,
        "instance_id": 22049,
        "index": 0,
        "pos